## 농아인 협회/센터 위치 수집

In [1]:
# 필요한 패키지를 불러옵니다.
import time
import json
import sqlite3
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'https://map.naver.com/v5/api/search'

In [3]:
params = {
    'caller': 'pcweb',
    'query': '농아인', # 농아인 검색
    'type': 'place',
    'searchCoord': '127.0406198501587;37.51741907323963',
    'page': '1',
    'displayCount': '20',
    'isPlaceRecommendationReplace': 'true',
    'lang': 'ko'
}

In [4]:
params['page'] = 1
resp = requests.get(url, params=params)
dom = BeautifulSoup(resp.text, 'html.parser')
json_obj = json.loads(resp.text)

In [ ]:
json_obj # 기관명, 분류, x좌표, y좌표, 주소가 필요

In [6]:
k = 1
second=[]
while True:
    params['page'] = k
    time.sleep(1)
    resp = requests.get(url, params=params)
    json_obj = json.loads(resp.text)

    for i in json_obj['result']['place']['list']:
        first=[]
        first.append(i['name'])
        first.append(i['category'])
        first.append(i['x'])
        first.append(i['y'])
        first.append(i['address'])

        second.append(first)

    k += 1
    
    if len(json_obj['result']['place']['list']) == 0:
        break

In [ ]:
df_1 = pd.DataFrame(second) # name, category, x, y, address가 변수로
category = df_1[1].values # 1번째 변수 = category
category = category.tolist()
category # 각 리스트마다 하나씩

In [ ]:
for i in range(len(category)):
    element = category[i]
    ctg = ""
    for j in range(len(element)):
        ctg += element[j]
        if j != (len(element) - 1):
            ctg +=","
    category[i] = ctg
category

In [9]:
df_1["Category"] = category

In [10]:
del df_1[1]

In [11]:
df_1.columns = ["Name", "Longitude", "Latitude", "Address", "Category"]
order = ["Name", "Category", "Longitude", "Latitude", "Address"]
df_1 = df_1[order]

In [12]:
conn = sqlite3.connect('naver_map.db')
cur = conn.cursor()

In [13]:
cur.executescript(''' 
    DROP TABLE IF EXISTS naver_map;
''')

In [14]:
conn.commit()
cur = conn.cursor()

In [15]:
df_1.to_sql('naver_map', conn)
cur.close()

## 협회/센터 위치 지도로 표시하기

In [ ]:
# 필요한 패키지를 불러옵니다.
import folium
import sqlite3
import pandas as pd
from flask import Flask
from folium import Marker
from folium.plugins import MarkerCluster

In [ ]:
# 저장된 DB를 불러옵니다.
conn = sqlite3.connect('naver_map.db')
df_1 = pd.read_sql("SELECT * FROM naver_map", conn)
conn.close()

In [ ]:
df_1

In [ ]:
app = Flask(__name__)

@app.route('/')
def fomap():
    m = folium.Map(location=[37.55, 127])
    mc = MarkerCluster()

    for i in range(len(df_1)):
        mc.add_child(
            Marker(location=[df_1.loc[i]['Latitude'],df_1.loc[i]['Longitude']],
                  popup=df_1.loc[i]['Name']))

    m.add_child(mc)
    return m.get_root().render()

if __name__ == '__main__':
    app.run()

# 국립국어원 수어사전 수집

In [16]:
# 필요한 패키지를 불러옵니다.
import re
import requests
from bs4 import BeautifulSoup
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [17]:
url = 'http://sldict.korean.go.kr/front/sign/signList.do'

In [18]:
params = {
    'current_pos_index': '',
    'origin_no': '0',
    'searchWay': '',
    'top_category': '',
    'category': 'SPE001', # SPE001: 법률 용어, SPE002: 교통, ...
    'detailCategory': '',
    'searchKeyword': '',
    'pageIndex': '1'
}

In [ ]:
params['pageIndex'] = 1
params['category'] = 'SPE001'
resp = requests.get(url, params=params,verify=False)
dom = BeautifulSoup(resp.text, 'html.parser')
dom.prettify

In [20]:
# 단어 카테고리 추출
dom.select('#menu > div > div > ul > li.on > ul > li > a > span')[0].text

'법률'

In [21]:
# 단어 제목 추출
title = dom.select('#list > li > div > p > span.tit > a')[0].text
re.sub('[^가-힣]','',title)

'취소권'

In [22]:
# 단어 뜻 추출
mean = dom.select('#list > li > div > p > span.info > a > cite > span')[0].text
re.sub('[\r\n\t]','',mean)

'[명사]의사 표시 및 법률 행위를 취소할 수 있는 권리.'

In [23]:
# 단어 영상 추출
s = dom.select('#signListForm > div.result_list.mt_30 > div.wrap_list > ul > li > div.list_left > div > a > img')[1].get('src')
re.findall('MOV.+',s)[0].replace('215X161.jpg', '700X466.mp4')

# s = s.replace('215X161.jpg', '700X466.mp4')
# re.findall('MOV.+',s)[0]

'MOV000266437_700X466.mp4'

In [24]:
# 크롤링 시작
url = 'http://sldict.korean.go.kr/front/sign/signList.do'

In [25]:
params = {
    'current_pos_index': '',
    'origin_no': '0',
    'searchWay': '',
    'top_category': '',
    'category': 'SPE001',
    'detailCategory': '',
    'searchKeyword': '',
    'pageIndex': '1'
}

In [26]:
category_num = 1
pageIndex_num = 1

category = '#menu > div > div > ul > li.on > ul > li > a > span'
title = '#list > li > div > p > span.tit > a'
mean = '#list > li > div > p > span.info > a > cite > span'
video = '#signListForm > div.result_list.mt_30 > div.wrap_list > ul > li > div.list_left > div > a > img'

while True:
    try:
        params['category'] = 'SPE'+str(category_num).zfill(3)
        while True:
            params['pageIndex'] = pageIndex_num
            resp = requests.get(url, params=params,verify=False)
            dom = BeautifulSoup(resp.text, 'html.parser')            
            for i in range(len(dom.select(title))):
                s = dom.select(video)[2*i+1].get('src')
                if len(re.findall('MOV.+',s.replace('215X161.jpg', '700X466.mp4'))) == 0:
                    continue
                f = open('add_category.txt','a')  
                f.write('\n'+dom.select(category)[category_num-1].text+
                        '\t'+dom.select(video)[2*i+1].get('src').replace('215X161.jpg', '700X466.mp4')+
                        '\t'+re.sub('[^가-힣]','',dom.select(title)[i].text)+
                        '\t'+re.sub('[\r\n\t]','',dom.select(mean)[i].text)+
                        '\t'+re.findall('MOV.+',s.replace('215X161.jpg', '700X466.mp4'))[0])
            if len(dom.select(title)) == 0:
                print('페이지 끝 \n')
                break
            pageIndex_num += 1
        category_num += 1
        pageIndex_num = 1
    except:
        print('카테고리 끝')
        f.close()
        break

카테고리 끝


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome()

In [ ]:
url = 'http://sldict.korean.go.kr/front/sign/signList.do?'
driver.get(url)

In [ ]:
i = 1
j = 0
category_list = ['CTE','SPE'] # CTE: 사회생활, SPE: 전문용어
category = category_list[j]
down_list = []
title_list = []

In [ ]:
while True:
    page = str(i)
#     print("i",page) # 페이지 확인
    url = 'http://sldict.korean.go.kr/front/sign/signList.do?top_category='+category+'&pageIndex='+page
    driver.get(url)
    time.sleep(1)
    num_content = driver.find_elements(By.XPATH,'//*[@id="list"]/li/div/p[1]/span[1]/a')
#     print(len(num_content)) # 페이지 내 갯수 확인
    
    if len(num_content) == 0:
        j += 1
        if j == 2:
            break
        category = category_list[j]
        i = 1
        continue
    
    for k in range(len(num_content)):
        driver.find_elements(By.XPATH,'//*[@id="list"]/li/div/p[1]/span[1]/a')[k].click()
        time.sleep(1)
        # 들어간 후 제목과 영상 추출
        down_url = driver.find_element(By.XPATH,'//*[@id="html5Video"]/source[2]').get_attribute('src')
        title = driver.find_element(By.XPATH,'//*[@id="signViewForm"]/dl/dd[1]').text
        title = re.sub('[^가-힣]','',title)
        print(down_url, title)
        down_list.append(down_url)
        title_list.append(title)
        driver.back()
        time.sleep(1)
            
    i += 1

## 한국 농아인협회 공지사항 수집

In [27]:
# 필요한 패키지를 불러옵니다.
import re
import sqlite3
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [28]:
url = 'http://www.deafkorea.com/sub_customer/notice.php'

In [29]:
params = {
    'b_name': 'notice',
    'code': '',
    'keyfield': '',
    'key': '',
    'page': ''
}

In [ ]:
params['page'] = 2
resp = requests.get(url, params=params)
resp.encoding = 'utf8'
dom = BeautifulSoup(resp.content, 'html.parser')
dom

In [31]:
# 공지사항 날짜
dom.select('table:not(.notice) td.w_date')

[<td class="w_date">2023.07.24</td>,
 <td class="w_date">2023.07.15</td>,
 <td class="w_date">2023.07.14</td>,
 <td class="w_date">2023.07.11</td>,
 <td class="w_date">2023.07.11</td>,
 <td class="w_date">2023.07.10</td>,
 <td class="w_date">2023.07.05</td>,
 <td class="w_date">2023.06.30</td>,
 <td class="w_date">2023.06.20</td>,
 <td class="w_date">2023.06.16</td>]

In [ ]:
# 공지사항 제목과 주소
dom.select('table:not(.notice) a')

In [ ]:
page_num = 1
c = []

while True:
    params['page'] = page_num
    resp = requests.get(url, params=params)
    resp.encoding = 'utf8'
    dom = BeautifulSoup(resp.content, 'html.parser')
    for a, b in zip(dom.select('table:not(.notice) a'), dom.select('table:not(.notice) td.w_date')):
        print('title:',a.text,'\ndate:',b.text,'\n',url+a['href'],'\n')
        c.extend([re.sub('[\r\n]','',a.text), b.text, url+a['href']])
    page_num += 1
    if page_num == 3:
        break

In [34]:
# 제목, 날짜, 주소 3개의 열을 가진 데이터프레임 만들기
n = 3
df_1 = [c[i*n : (i+1)*n] for i in range((len(c) + n - 1) // n )] 
df = pd.DataFrame.from_records(df_1,columns=('title','date','url'))

In [35]:
conn = sqlite3.connect('sonmin.db')
cur = conn.cursor()

In [36]:
cur.executescript(''' 
    DROP TABLE IF EXISTS sonmin;
    CREATE TABLE sonmin(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    title TEXT  NOT NULL,
    date  TEXT   NOT NULL,
    url TEXT NOT NULL
    );
''')

In [37]:
conn.commit()
cur = conn.cursor()

In [38]:
sql = "INSERT INTO sonmin(title, date, url) VALUES(?,?,?)"
cur.executemany(sql, df_1) # 제목, 날짜, 주소 순으로 DB에 넣기

In [39]:
conn.commit()
cur.close()

In [40]:
conn = sqlite3.connect('sonmin.db')
cur = conn.cursor()

In [ ]:
# 최신순으로 정렬
cur.execute("SELECT * FROM sonmin ORDER BY strftime('%Y-%m-%d', date)")
cur.fetchall()

In [42]:
conn.commit()
cur.close()